In [16]:
import os
import re
import numpy as np
import tifffile as tiff
import cv2
from skimage import io

In [17]:
def process_images(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok = True)
    
   # pattern = re.compile(r"R\d{2}-C\d{2}-F\d+_(\d{5})_(IM|CM)_(KIF11_orig|LUC_orig)")
    
    for filename in os.listdir(input_folder):
        if not filename.endswith(".tif"):
            continue

        output_path = os.path.join(output_folder, filename)
        # Skip processing if the output file already exists
        if os.path.exists(output_path):
            print(f"Skipping {filename}, already processed.")
            continue
        
        input_path = os.path.join(input_folder, filename)
        with tiff.TiffFile(input_path) as tif_file:
            img = tif_file.asarray()
            print(f"The dimensions of the input image are {img.shape}")
            metadata = tif_file.pages[0].tags
            resolution_tag = metadata.get('XResolution')
            z_resolution_tag = metadata.get('ZResolution')
            pixel_size = 0.1883734  # Default XY pixel size
            z_pixel_size = 1.0  # Default Z pixel size
            
            if resolution_tag:
                resolution = resolution_tag.value
                pixel_size = resolution[1] / resolution[0]
            
            if z_resolution_tag:
                z_resolution = z_resolution_tag.value
                z_pixel_size = z_resolution[1] / z_resolution[0]
        
        if len(img.shape) == 4 and img.shape[1] == 4:  # (Z, C, H, W)
            img = np.moveaxis(img, 1, 0)  # (C, Z, H, W)
            print(f"The dimensions of the image after np.moveaxis: {img.shape}")
            ch3_proj = np.max(img[1], axis = 0)  # Max project Ch3 (Ch2 in this set)
            
            _, thresh = cv2.threshold(ch3_proj, 0, 255, cv2.THRESH_OTSU)
            contours, _ = cv2.findContours(thresh.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            objects = sorted(contours, key = cv2.contourArea, reverse = True)[:1]  # Get up to 1 largest objects
            
            for idx, cnt in enumerate(objects):
                x, y, w, h = cv2.boundingRect(cnt)
                center_x, center_y = x + w // 2, y + h // 2
                
                half_size = 75  # 150x150 crop
                x1, x2 = center_x - half_size, center_x + half_size
                y1, y2 = center_y - half_size, center_y + half_size
                
                pad_x1, pad_x2, pad_y1, pad_y2 = 0, 0, 0, 0
                if x1 < 0:
                    pad_x1 = abs(x1)
                    x1 = 0
                if x2 > img.shape[3]:
                    pad_x2 = x2 - img.shape[3]
                    x2 = img.shape[3]
                if y1 < 0:
                    pad_y1 = abs(y1)
                    y1 = 0
                if y2 > img.shape[2]:
                    pad_y2 = y2 - img.shape[2]
                    y2 = img.shape[2]
                
                cropped_ch2 = img[3, :, y1:y2, x1:x2] #CZYX TUBULIN
                cropped_ch3 = img[1, :, y1:y2, x1:x2] #CZYX PHH3

                print(f"Shape of cropped_ch2 before padding: {cropped_ch2.shape}")
                print(f"Shape of cropped_ch3 before padding: {cropped_ch3.shape}")
                
                cropped_ch2 = np.pad(cropped_ch2, ((0, 0), (pad_y1, pad_y2), (pad_x1, pad_x2)), mode = 'constant', constant_values = 0)
                cropped_ch3 = np.pad(cropped_ch3, ((0, 0), (pad_y1, pad_y2), (pad_x1, pad_x2)), mode = 'constant', constant_values = 0)
                
                cropped_ch2 = np.pad(cropped_ch2, ((1, 1), (0, 0), (0, 0)), mode = 'constant', constant_values = 0)  # Add empty Z slices
                cropped_ch3 = np.pad(cropped_ch3, ((1, 1), (0, 0), (0, 0)), mode = 'constant', constant_values = 0)  # Add empty Z slices

                print(f"Shape of cropped_ch2: {cropped_ch2.shape}")
                print(f"Shape of cropped_ch3: {cropped_ch3.shape}")
                
                # Stack channels together in (Z, C, Y, X) format
                cropped_stack = np.stack([cropped_ch2, cropped_ch3], axis = 1)  # (Z, C, Y, X)
                cropped_stack = np.expand_dims(cropped_stack, axis = 0) # (T, Z, C, Y, X)
                                
                print(f"Shape of cropped_stack after merging and expansion: {cropped_stack.shape}")
                
                output_filename = os.path.join(output_folder, f"{filename.replace('.tif', '')}_obj{idx+1}_crop.tif")
                if os.path.exists(output_filename):
                    print(f"Skipping {output_filename}, already processed.")
                    continue

                tiff.imwrite(
                    output_filename, 
                    cropped_stack, 
                    imagej = True,
                    metadata = {
                        'spacing': 1, #This is hardcoded for now, TODO read from imput!
                        'unit': 'um',
                        'axes': 'TZCYX'
                    },
                    resolution = (1.0 / pixel_size, 1.0 / pixel_size)
                )
                print(f"Saved: {output_filename}")
    print("Done.")

In [ ]:
input_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250314/01_Importer"
output_folder = "/Volumes/arxivBeta/_Tobias/Opera/20250314/03_Crop"
process_images(input_folder, output_folder)

The dimensions of the input image are (16, 4, 1080, 1080)
The dimensions of the image after np.moveaxis: (4, 16, 1080, 1080)
The dimensions of the input image are (16, 4, 1080, 1080)
The dimensions of the image after np.moveaxis: (4, 16, 1080, 1080)
The dimensions of the input image are (16, 4, 1080, 1080)
The dimensions of the image after np.moveaxis: (4, 16, 1080, 1080)
The dimensions of the input image are (16, 4, 1080, 1080)
The dimensions of the image after np.moveaxis: (4, 16, 1080, 1080)
The dimensions of the input image are (16, 4, 1080, 1080)
The dimensions of the image after np.moveaxis: (4, 16, 1080, 1080)
The dimensions of the input image are (16, 4, 1080, 1080)
The dimensions of the image after np.moveaxis: (4, 16, 1080, 1080)
The dimensions of the input image are (16, 4, 1080, 1080)
The dimensions of the image after np.moveaxis: (4, 16, 1080, 1080)
The dimensions of the input image are (16, 4, 1080, 1080)
The dimensions of the image after np.moveaxis: (4, 16, 1080, 1080)
